In [1]:
#导包
from netCDF4 import Dataset
from tensorflow.keras import layers, models
from tensorflow.keras.layers import ConvLSTM2D,LSTM, BatchNormalization, LayerNormalization,Input, Conv3D, TimeDistributed, Flatten
from tensorflow.keras.layers import Concatenate,Conv2D,TimeDistributed, MaxPooling2D, Input, MaxPooling3D
from tensorflow.keras.layers import  Reshape,multiply
from tensorflow.keras.layers import Layer,Lambda,Dot,ReLU, Dense, Dropout, Activation, Flatten,Attention
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.optimizers import Adam,SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.losses import MeanSquaredError
from sklearn.metrics import r2_score,accuracy_score,precision_score
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras import regularizers
import tensorflow as tf
from datetime import datetime
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import models, layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import tensorflow.keras as keras
from tensorflow.keras import layers, Sequential, regularizers
# calculate RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt
import pandas as pd 
import tensorflow as tf
import gc 
import gzip
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import match
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os 
%matplotlib inline

In [2]:
# 检查可用GPU数量
len(tf.config.experimental.list_physical_devices('GPU'))

1

# 读取数据

## SSH

In [3]:
data_ssh = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/SSH/SSH_2005-2015_month.nc')

In [4]:
time_ssh = data_ssh['time'][:]
print(len(time_ssh))

132


In [5]:
# 查看研究区域经纬度
ssh_lat = data_ssh['lat'][:].data
#print(ssh_lat[:])
ssh_lon = data_ssh['lon'][:].data
#print(ssh_lon[:])

In [6]:
ssh = data_ssh['ssh'][:].data
print(ssh.shape)

(132, 280, 600)


In [7]:
# 查看最大值与最小值
np.nanmax(ssh),np.nanmin(ssh)

(2.20051, -0.43673334)

## SSS

In [8]:
data_sss = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/SSS/dataset-sss-ssd-rep-monthly_2005-2020.nc')

In [9]:
# 确定时间
time_sss = data_sss['time'][:132]
print(len(time_sss))

132


In [10]:
# 获取研究位置经纬度
sss_lat = data_sss['lat'][:].data
#print(sss_lat[:])
sss_lon = data_sss['lon'][:].data
#print(sss_lon[:])

In [11]:
sss = data_sss['sos'][:132].data 
print(sss.shape)
sss = np.squeeze(sss) # 移除大小为一的维度
print(sss.shape)

(132, 1, 280, 600)
(132, 280, 600)


In [12]:
# 查看最大值，最小值
np.nanmax(sss),np.nanmin(sss)

(36.072178, 13.604756)

## SSW

In [13]:
data_ssw = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/SSW/anon_ssw_2005_2018_Month_data.nc')

In [14]:
# 确定时间
time_ssw = data_ssw['time'][:132]
print(len(time_ssw))

132


In [15]:
lat_ssw = data_ssw['lat'][:].data
#print(lat_ssw[314:594])
lon_ssw = data_ssw['lon'][:].data
#print(lon_ssw[440:1040])

# lat 0.125    ----    69.875
# lon  110.125    ----    259.875

In [16]:
uwnd = data_ssw['uwnd'][:132,314:594,440:1040].data  
vwnd = data_ssw['vwnd'][:132,314:594,440:1040].data

In [17]:
print(uwnd.shape)
print(vwnd.shape)

(132, 280, 600)
(132, 280, 600)


In [18]:
np.nanmax(uwnd),np.nanmin(uwnd),np.nanmax(vwnd),np.nanmin(vwnd)

(12.433594, -11.7988205, 8.8359995, -13.645481)

## SST 海表面温度

In [19]:
data_3dt_1 = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/3DT/dataset-armor-3d-rep-monthly_2005-2009.nc')
data_3dt_2 = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/3DT/dataset-armor-3d-rep-monthly_2010-2015.nc')

In [20]:
sst_1 = data_3dt_1['to'][:].data
sst_2 = data_3dt_2['to'][:].data

In [21]:
print(sst_1.shape)
print(sst_2.shape)

(60, 43, 280, 600)
(72, 43, 280, 600)


In [22]:
temp_depth = np.append(sst_1,sst_2,axis=0)
print(temp_depth.shape)

(132, 43, 280, 600)


In [23]:
sst = temp_depth[:,0:1,:,:]
sst = np.squeeze(sst)
print(sst.shape)

(132, 280, 600)


## 3DS

In [24]:
data_3ds = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/3DT/sali/3DS_end_2005_2015.nc')
sali_depth = data_3ds['so'][:].data
print(sali_depth.shape)

(132, 43, 280, 600)


In [25]:
# 所有深度层
depths = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
         450,500,550,600,700,800,900,1000,1100,1200,1300,1400,1500,1750,2000]

#选取的深度层
depths_use = [10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
              500,600,700,800,900,1000,1100,1300,1500,1750,2000]

#[50,100,125,150,200,250,300,400,500,600,700,800,900,1000]

#[5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,500,600,700,800,900,1000,1100,1300,1500,1750,2000]
# 1 2  3  4  5  6  7  8  9 10  11  12  13  14  15  16  17  18 19  20  21   22  23  24  25  26  27  28   29   30   31    32   33

In [26]:
# 使用5m深度的盐度给sali_depth_use 赋值
sali_depth_use = sali_depth[:,1:2]

In [27]:
for i,depth in enumerate(depths):
    if(depth in depths_use):
        #print(depths[i])
        sali_depth_use = np.concatenate((sali_depth_use,sali_depth[:,i:i+1]),axis=1)  

In [28]:
print(sali_depth_use.shape)
sali_depth_use = np.transpose(sali_depth_use, (0,2,3,1))
print(sali_depth_use.shape)

(132, 33, 280, 600)
(132, 280, 600, 33)


## 数据合并

In [29]:
print(ssh.shape,sst.shape,uwnd.shape,vwnd.shape,sss.shape)
cat_data =np.stack((ssh,sst,uwnd,vwnd,sss),axis=3)
cat_data.shape

(132, 280, 600) (132, 280, 600) (132, 280, 600) (132, 280, 600) (132, 280, 600)


(132, 280, 600, 5)

In [30]:
# 将表面数据和水下数据在最后一个维度拼接，将空值删除。
cat_data_ss_depth = np.concatenate((cat_data,sali_depth_use),axis=-1)

In [31]:
cat_data_ss_depth.shape 

(132, 280, 600, 38)

## 将填充值赋值为NAN，且一个点中有一个维度为nan，则将这个点的所有维度都设置为nan 


In [32]:
## 先对合并的数据进行reshape成（长*宽，时间*通道），此步骤是为了统一图像， 让所有时间所有深度的图像都相同。
cat_data_ss_depth_reshape  = np.transpose(cat_data_ss_depth,(1,2,0,3))
print(cat_data_ss_depth_reshape.shape)
cat_data_ss_depth_reshape = cat_data_ss_depth_reshape.reshape ((-1,132*38))
print(cat_data_ss_depth_reshape.shape)

(280, 600, 132, 38)
(168000, 5016)


In [33]:
# 将填充值赋值为nan
cat_data_ss_depth_reshape[cat_data_ss_depth_reshape == 32767] = np.nan
nan_mask = np.isnan(cat_data_ss_depth_reshape)

In [34]:
# 查看nan的个数 63271
np.sum(nan_mask.any(axis=1) == True)

63271

In [35]:
# 通过布尔索引删除NaN值。 nan_mask.any(axis=1)==true,只要第一个维度中含有nan就将所有设置为nan。
cat_data_ss_depth_reshape[nan_mask.any(axis=1) == True] = np.nan   
print('cat_data_ss_depth_reshape:',cat_data_ss_depth_reshape.shape)

cat_data_ss_depth_reshape: (168000, 5016)


In [36]:
# 查看nan的个数，共有64471。
np.sum(np.isnan(cat_data_ss_depth_reshape[:,4])) # 此处写几都可以，得到的nan的数量都是相同的。 

63271

In [37]:
cat_data_ss_depth_reshape = cat_data_ss_depth_reshape.reshape(-1,132,38)
print(cat_data_ss_depth_reshape.shape)
cat_data_ss_depth_reshape = np.transpose(cat_data_ss_depth_reshape,(1,0,2))
print(cat_data_ss_depth_reshape.shape)
cat_data_ss_depth_reshape = cat_data_ss_depth_reshape.reshape(-1,280,600,38)
print(cat_data_ss_depth_reshape.shape)

(168000, 132, 38)
(132, 168000, 38)
(132, 280, 600, 38)


In [38]:
# 确认nan的个数
np.sum(np.isnan(cat_data_ss_depth_reshape[1:2,:,:,2:3]))

63271

## 将数据分成小块

In [39]:
cat_data_ss = cat_data_ss_depth_reshape[:,:,:,:5]    #表面数据
cat_data_depth = cat_data_ss_depth_reshape[:,:,:,5:]  #次表层数据
print(cat_data_ss.shape,cat_data_depth.shape)

(132, 280, 600, 5) (132, 280, 600, 33)


In [40]:
np.sum(np.isnan(cat_data_depth[1:2,:,:,5:6]))

63271

In [41]:
import numpy as np
import pandas as pd

time_size, lat_size, lon_size, channel = cat_data_ss.shape   # （132，280，600，5）

# 定义矩形框的大小和步幅           
box_size = 9  # 单位为度
step_size = 1  # 滑动步幅

# 计算输出网格的形状
out_lat_size = (lat_size - box_size) // step_size  + 1 
out_lon_size = (lon_size - box_size) // step_size  + 1 

print('time_size:',time_size,'lat_size:',lat_size,'lon_size:',lon_size)   #(132 280 600)
print(out_lat_size,out_lon_size)  #(272 592)

#输出数组 
out_data = np.zeros((time_size, out_lat_size , out_lon_size, box_size, box_size, channel)) #(132, 272, 592, 9,9,5)
out_data_y = np.zeros((time_size, out_lat_size , out_lon_size, 33)) #(132, 272, 592,15)

# 遍历时间步、纬度和经度
for t in range(time_size):
    sample_idx = 0
    for lat in range(0, lat_size - box_size +1, step_size):
        for lon in range(0, lon_size - box_size +1, step_size):
            # 计算当前矩形框的边界
            lat_min = lat
            lat_max = lat + box_size
            lon_min = lon
            lon_max = lon + box_size
            #print(lat_min,lat_max,lon_min,lon_max)
            # 获取当前矩形框的地图数据
            box_data = cat_data_ss[t, lat_min:lat_max, lon_min:lon_max,:]
            # 写入数组中
#             lat_ = lat//4
#             lon_ = lon//4
            out_data[t,lat,lon, :, :, :] = box_data
            box_data = []
            #print(sample_idx)
            # 增加样本计数器
            sample_idx += 1

print('out_data的形状',out_data.shape)
#return out_data
    
#==============================================================================#   

# 遍历时间步、纬度和经度
for t in range(time_size):
    sample_idx = 0
    for lat in range(4, lat_size-4, step_size):
        for lon in range(4, lon_size-4, step_size):
            box_data_y = cat_data_depth[t, lat:lat+1, lon:lon+1,:]
            out_data_y[t,lat-4,lon-4, :] = box_data_y
            box_data_y = []
            #print(sample_idx)
            # 增加样本计数器
            sample_idx += 1

print('out_data_y的形状',out_data_y.shape)
    
# out_data_reshape = slider_value(cat_data)

time_size: 132 lat_size: 280 lon_size: 600
272 592
out_data的形状 (132, 272, 592, 9, 9, 5)
out_data_y的形状 (132, 272, 592, 33)


## 删除缺失值多于一半的数据

### 获取NAN_mask 

In [42]:
out_data.shape 

(132, 272, 592, 9, 9, 5)

In [43]:
# 修改变量形状
out_data_reshape = out_data.reshape((132,-1,9,9,5))
print(out_data_reshape.shape) 

# 获取变量nan掩码
nan_mask_out_data = np.isnan(out_data_reshape)
print(nan_mask_out_data.shape)

(132, 161024, 9, 9, 5)
(132, 161024, 9, 9, 5)


In [44]:
# （161024，9，9）中存在4666932个nan
np.count_nonzero(nan_mask_out_data[7:8,:,:,:,2:3])

4764087

In [45]:
# 选取一个作为代表(都一样)
nan_mask_out_data_one = nan_mask_out_data[0,:,:,:,0]   #(161024, 9, 9)
nan_mask_out_data_one = nan_mask_out_data_one.reshape(-1,9*9) #（161024，81）    # 其中true 代表Nan

In [46]:
# 查看mask中Nan 的个数
#np.count_nonzero(nan_mask_out_data_one)     # 4666932
#查看形状
#nan_mask_out_data_one.shape     # 161024, 81)
# 查看内容
#nan_mask_out_data_one

### 为有效值大于一半的方块标注True

In [47]:
nan_mask = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <=40 and i[40] == False):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask.append(True)   # True 代表保留
    else:
        nan_mask.append(False) # False 代表删除

In [48]:
np.count_nonzero(nan_mask)  # 查看有效值个数

101799

In [49]:
# 修改形状，将lat*lon 放在前面，用于删除无效值
out_data_reshape =  np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y,(1,2,0,3))
print('out_data_y_reshape:',out_data_y_reshape.shape)
out_data_y_reshape = out_data_y_reshape.reshape(-1,132,33)
print('out_data_y_reshape:',out_data_y_reshape.shape)

out_data_reshape: (161024, 132, 9, 9, 5)
out_data_y_reshape: (272, 592, 132, 33)
out_data_y_reshape: (161024, 132, 33)


## 开始删除无效数据

In [50]:
out_data_reshape = out_data_reshape[nan_mask]
out_data_y_reshape = out_data_y_reshape[nan_mask]
print(out_data_reshape.shape,out_data_y_reshape.shape)

(101799, 132, 9, 9, 5) (101799, 132, 33)


## 使用0来填充nan 

In [51]:
out_data_reshape = np.nan_to_num(out_data_reshape, nan=0)  
out_data_y_reshape = np.nan_to_num(out_data_y_reshape, nan=0)  

# 验证集和测试集的划分

## 对数据进行reshape,以进行数据集的划分

In [52]:
# 将时间维度放到前面，用于数据集的划分
out_data_reshape = np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y_reshape,(1,0,2))
print('out_data_y_reshape:',out_data_y_reshape.shape)

out_data_reshape: (132, 101799, 9, 9, 5)
out_data_y_reshape: (132, 101799, 33)


In [53]:
# 最后十二个月的数据作为test
test_num = 120
x_train = out_data_reshape[:test_num]
y_train = out_data_y_reshape[:test_num]

x_test = out_data_reshape[test_num:]
y_test = out_data_y_reshape[test_num:]

In [54]:
# 查看形状
x_train.shape,y_train.shape,x_test.shape,y_test.shape 

((120, 101799, 9, 9, 5),
 (120, 101799, 33),
 (12, 101799, 9, 9, 5),
 (12, 101799, 33))

In [55]:
# 将前两个维度进行合并  时间*(lat*lon)
x_train = np.reshape(x_train,(-1,9,9,5))
x_test = np.reshape(x_test,(-1,9,9,5))

y_train = np.reshape(y_train,(-1,33))
y_test = np.reshape(y_test,(-1,33))

## 划分训练数据和目标数据

In [56]:
print('x_test.shape:',x_test.shape)
print('y_test.shape',y_test.shape)
print('x_train',x_train.shape)
print('y_train',y_train.shape)

x_test.shape: (1221588, 9, 9, 5)
y_test.shape (1221588, 33)
x_train (12215880, 9, 9, 5)
y_train (12215880, 33)


In [57]:
# 这是验证集和训练集的划分是随机选取。
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=100)

In [58]:
print('x_train',x_train.shape)
print('x_test',x_test.shape)
print('x_val',x_val.shape)
print('y_train',y_train.shape)
print('y_test',y_test.shape)
print('y_val',y_val.shape)

x_train (8551116, 9, 9, 5)
x_test (1221588, 9, 9, 5)
x_val (3664764, 9, 9, 5)
y_train (8551116, 33)
y_test (1221588, 33)
y_val (3664764, 33)


# 特征归一化

In [59]:
# 记录数据集的形状
x_train_shape = x_train.shape
y_train_shape = y_train.shape

x_test_shape = x_test.shape
y_test_shape = y_test.shape

x_val_shape = x_val.shape
y_val_shape = y_val.shape

In [60]:
scaler_f = StandardScaler()
x_train = scaler_f.fit_transform(x_train.reshape(-1,x_train_shape[3])) 
x_test = scaler_f.transform(x_test.reshape(-1,x_test_shape[3]))
x_val = scaler_f.transform(x_val.reshape(-1,x_val_shape[3]))

scaler_l = StandardScaler()
y_train = scaler_l.fit_transform(y_train.reshape(-1,y_train_shape[1])) 
y_test = scaler_l.transform(y_test.reshape(-1,y_test_shape[1])) 
y_val = scaler_l.transform(y_val.reshape(-1,y_val_shape[1])) 

In [61]:
x_train = np.reshape(x_train,(-1,9,9,5))
x_test = np.reshape(x_test,(-1,9,9,5))
x_val = np.reshape(x_val,(-1,9,9,5))
y_train = np.reshape(y_train,(-1,33))
y_test = np.reshape(y_test,(-1,33))
y_val = np.reshape(y_val,(-1,33))

In [62]:
print('x_train',x_train.shape)
print('x_test',x_test.shape)
print('x_val',x_val.shape)
print('y_train',y_train.shape)
print('y_test',y_test.shape)
print('y_val',y_val.shape)

x_train (8551116, 9, 9, 5)
x_test (1221588, 9, 9, 5)
x_val (3664764, 9, 9, 5)
y_train (8551116, 33)
y_test (1221588, 33)
y_val (3664764, 33)


# 定义网络结构

In [63]:
# # 创建神经网络模型
# def create_model():   
#     model = models.Sequential([
#         layers.Dense(200, activation='tanh', input_shape=(5,)),  # 输入层和隐藏层
#         layers.Dense(15, activation='tanh'),
#         layers.Dense(8, activation='tanh'),
#         layers.Dense(1, activation='linear')  # 输出层，num_depths表示深度的数量
#     ])
#     model.compile(optimizer='adam', loss='mean_squared_error')
#     return model 
# #model.summary()

In [64]:
#  通道注意力 #
class ChannelAttention_CNN(layers.Layer):
    def __init__(self, in_planes, ratio=2):  #修改部分：由2修改为了4
        super(ChannelAttention_CNN, self).__init__()
        self.in_planes = in_planes
        self.ratio = ratio
        self.avg_out= layers.GlobalAveragePooling2D()
        self.max_out= layers.GlobalMaxPooling2D()

        self.fc1 = layers.Dense(in_planes//ratio, kernel_initializer='he_normal',
                                kernel_regularizer=regularizers.l2(5e-4),
                                activation=tf.nn.relu,
                                use_bias=True, bias_initializer='zeros')
        self.fc2 = layers.Dense(in_planes, kernel_initializer='he_normal',
                                kernel_regularizer=regularizers.l2(5e-4),
                                use_bias=True, bias_initializer='zeros')

    def call(self,inputs):
        avg_out = self.avg_out(inputs)
        max_out = self.max_out(inputs)
        out = tf.stack([avg_out, max_out], axis=1)  # shape=(None, 2, fea_num)
        out = self.fc2(self.fc1(out))
        out = tf.reduce_sum(out, axis=1)             # shape=(256, 512)
        out = tf.nn.sigmoid(out)
        out = layers.Reshape((1, 1, out.shape[1]))(out)
        return out*inputs
    
    def get_config(self):
        return{'ratio': self.ratio,
                'in_planes': self.in_planes } 

    
# 空间注意力 #
def regularized_padded_conv(*args, **kwargs):
    return layers.Conv2D(*args, **kwargs, padding='same', use_bias=False,
                         kernel_initializer='he_normal',
                         kernel_regularizer=regularizers.l2(5e-4))

class SpatialAttention_CNN(layers.Layer):
    def __init__(self, kernel_size=5):
        super(SpatialAttention_CNN, self).__init__()
        self.kernel_size = kernel_size
        self.conv1 = regularized_padded_conv(1, kernel_size=kernel_size, strides=1, activation='sigmoid')
    def call(self,inputs):
        avg_out = tf.reduce_mean(inputs, axis=3)
        max_out = tf.reduce_max(inputs, axis=3)
        out = tf.stack([avg_out, max_out], axis=-1)             # 创建一个维度,拼接到一起concat。
        out = self.conv1(out)
        return out*inputs
    
    def get_config(self):
        return{'kernel_size': self.kernel_size} 

In [65]:
# def ChannelAttention(inputs, in_planes, ratio=2):
    
#     avg_out= layers.GlobalAveragePooling2D()
#     max_out= layers.GlobalMaxPooling2D()

#     fc1 = layers.Dense(in_planes//ratio, kernel_initializer='he_normal',
#                             kernel_regularizer=regularizers.l2(5e-4),
#                             activation=tf.nn.relu,
#                             use_bias=True, bias_initializer='zeros')
#     fc2 = layers.Dense(in_planes, kernel_initializer='he_normal',
#                             kernel_regularizer=regularizers.l2(5e-4),
#                             use_bias=True, bias_initializer='zeros')

    
#     avg_out = avg_out(inputs)
#     max_out = max_out(inputs)
#     out = tf.stack([avg_out, max_out], axis=1)  # shape=(None, 2, fea_num)
#     out = fc2(fc1(out))
#     out = tf.reduce_sum(out, axis=1)             # shape=(256, 512)
#     out = tf.nn.sigmoid(out)
#     out = layers.Reshape((1, 1, out.shape[1]))(out)

#     return  out*inputs

In [66]:
# def regularized_padded_conv(*args, **kwargs):
#     return layers.Conv2D(*args, **kwargs, padding='same', use_bias=False,
#                          kernel_initializer='he_normal',
#                          kernel_regularizer=regularizers.l2(5e-4))

# def SpatialAttention(inputs,kernel_size=5):
#     conv1 = regularized_padded_conv(1, kernel_size=kernel_size, strides=1, activation='sigmoid')
#     avg_out = tf.reduce_mean(inputs, axis=3)
#     max_out = tf.reduce_max(inputs, axis=3)
#     out = tf.stack([avg_out, max_out], axis=-1)             # 创建一个维度,拼接到一起concat。
#     out = conv1(out)
    
#     return out*inputs

In [67]:
# def cbam_block(inputs, in_planes, ratio=2):

#     cbam_feature = ChannelAttention(inputs, in_planes, ratio)
#     cbam_feature = SpatialAttention(cbam_feature)
#     return cbam_feature

In [68]:
def create_model():
    model_input= Input(shape=x_train.shape[1:])
    output_size = 1
    
    #卷积层一
    x = Conv2D(filters=16, kernel_size=(3, 3),strides=1)(model_input)
    #x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(rate=0.1)(x)

    x = ChannelAttention_CNN(16)(x)
    x = SpatialAttention_CNN()(x)
    
    #卷积层二
    x = Conv2D(filters=32, kernel_size=(3, 3),strides=1)(x)
    #x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(rate=0.1)(x)

    x = ChannelAttention_CNN(32)(x)
    x= SpatialAttention_CNN()(x)
    
    x = Flatten()(x)
    x = Dense(units=16)(x)
    x = Activation('relu')(x)

    out = Dense(units=output_size)(x)
    
    model = Model(inputs=model_input, outputs=out)
    return model 

In [69]:
model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 9, 9, 5)]         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 7, 7, 16)          736       
_________________________________________________________________
activation (Activation)      (None, 7, 7, 16)          0         
_________________________________________________________________
dropout (Dropout)            (None, 7, 7, 16)          0         
_________________________________________________________________
channel_attention_cnn (Chann (None, 7, 7, 16)          280       
_________________________________________________________________
spatial_attention_cnn (Spati (None, 7, 7, 16)          50        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 5, 32)          4640  

In [70]:
# initial_learning_rate = 0.0005
# optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)


# model.compile(optimizer='adam', loss='mean_squared_error')
# checkpoint_path='./CNN_model_30.h5'
# keras_callbacks   = [
#       EarlyStopping(monitor='val_loss', patience=30, mode='min', min_delta=0.001),
#       ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')
# ]

In [71]:
# num_epochs = 1000
# batch_size = 10240

# history = model.fit(x_train, y_train[:,14:15], validation_data=(x_val,y_val[:,14:15]),
#                     epochs=num_epochs, batch_size=batch_size, verbose=2, callbacks=keras_callbacks)

## 网格搜索交叉验证寻找最佳参数

In [72]:
# model = KerasRegressor(build_fn=create_model,verbose=2)

In [73]:
# param_grid = {
#     'neurons1': [16,32,64,128],
#     'neurons2': [32,64,128,256],
# }

In [74]:
# # 使用GridSearchCV进行交叉验证并搜索最佳参数组合：
# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
# grid_result = grid_search.fit(x_train, y_train[:,14:15],epochs=100,verbose=2,batch_size=10240)

In [75]:
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

## 训练

In [77]:
def train_model_CNN(x_train,y_train,x_val,y_val,name):
    # 创建网络
    model = create_model()
    # 编译网络
    model.compile(loss='mse', optimizer='adam')
    checkpoint_path=name
    keras_callbacks   = [
          EarlyStopping(monitor='val_loss', patience=16, mode='min', min_delta=0.001),
          ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')
    ]
    #ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, mode='min', min_delta=0.001),
    #TensorBoard(log_dir='./logs', histogram_freq=1)
    
    model.fit(x_train, y_train, validation_data=(x_val,y_val),
                    epochs=300, batch_size=10240, verbose=2, callbacks=keras_callbacks)

In [ ]:
gc.collect()

In [78]:
depth = [5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
              500,600,700,800,900,1000,1100,1300,1500,1750,2000]

#depth = [600,700,800,900,1000,1100,1300,1500,1750,2000]
# 训练30 - 1000m
for i in range(23,33):
    gc.collect()
    train_model_CNN(x_train,y_train[:,i:i+1],x_val,y_val[:,i:i+1],   
                'D:/codeFile/jupyterDemo/3d_ts/BPNN/model/new_0-2000m_24.6.6/CNN/sali/model4/CNN_sali_'+str(depth[i])+'m.h5')

Epoch 1/300
836/836 - 37s - loss: 0.1440 - val_loss: 0.1068
Epoch 2/300
836/836 - 22s - loss: 0.0738 - val_loss: 0.0816
Epoch 3/300
836/836 - 22s - loss: 0.0634 - val_loss: 0.0768
Epoch 4/300
836/836 - 22s - loss: 0.0580 - val_loss: 0.0802
Epoch 5/300
836/836 - 22s - loss: 0.0541 - val_loss: 0.0772
Epoch 6/300
836/836 - 22s - loss: 0.0510 - val_loss: 0.0655
Epoch 7/300
836/836 - 22s - loss: 0.0487 - val_loss: 0.0605
Epoch 8/300
836/836 - 22s - loss: 0.0468 - val_loss: 0.0648
Epoch 9/300
836/836 - 22s - loss: 0.0454 - val_loss: 0.0677
Epoch 10/300
836/836 - 22s - loss: 0.0443 - val_loss: 0.0670
Epoch 11/300
836/836 - 22s - loss: 0.0432 - val_loss: 0.0554
Epoch 12/300
836/836 - 22s - loss: 0.0423 - val_loss: 0.0600
Epoch 13/300
836/836 - 22s - loss: 0.0416 - val_loss: 0.0570
Epoch 14/300
836/836 - 22s - loss: 0.0409 - val_loss: 0.0550
Epoch 15/300
836/836 - 22s - loss: 0.0404 - val_loss: 0.0595
Epoch 16/300
836/836 - 22s - loss: 0.0399 - val_loss: 0.0541
Epoch 17/300
836/836 - 22s - loss

Epoch 60/300
836/836 - 22s - loss: 0.0371 - val_loss: 0.0568
Epoch 61/300
836/836 - 22s - loss: 0.0371 - val_loss: 0.0555
Epoch 62/300
836/836 - 22s - loss: 0.0370 - val_loss: 0.0516
Epoch 63/300
836/836 - 22s - loss: 0.0370 - val_loss: 0.0460
Epoch 64/300
836/836 - 22s - loss: 0.0369 - val_loss: 0.0416
Epoch 65/300
836/836 - 22s - loss: 0.0368 - val_loss: 0.0511
Epoch 66/300
836/836 - 22s - loss: 0.0368 - val_loss: 0.0499
Epoch 67/300
836/836 - 22s - loss: 0.0367 - val_loss: 0.0471
Epoch 68/300
836/836 - 22s - loss: 0.0367 - val_loss: 0.0524
Epoch 69/300
836/836 - 22s - loss: 0.0366 - val_loss: 0.0547
Epoch 70/300
836/836 - 22s - loss: 0.0366 - val_loss: 0.0464
Epoch 71/300
836/836 - 22s - loss: 0.0366 - val_loss: 0.0511
Epoch 72/300
836/836 - 22s - loss: 0.0365 - val_loss: 0.0541
Epoch 73/300
836/836 - 22s - loss: 0.0365 - val_loss: 0.0507
Epoch 74/300
836/836 - 22s - loss: 0.0364 - val_loss: 0.0464
Epoch 75/300
836/836 - 22s - loss: 0.0364 - val_loss: 0.0499
Epoch 76/300
836/836 - 2

Epoch 3/300
836/836 - 22s - loss: 0.0862 - val_loss: 0.0901
Epoch 4/300
836/836 - 22s - loss: 0.0782 - val_loss: 0.0875
Epoch 5/300
836/836 - 22s - loss: 0.0731 - val_loss: 0.1073
Epoch 6/300
836/836 - 22s - loss: 0.0694 - val_loss: 0.0850
Epoch 7/300
836/836 - 22s - loss: 0.0664 - val_loss: 0.0790
Epoch 8/300
836/836 - 22s - loss: 0.0641 - val_loss: 0.0771
Epoch 9/300
836/836 - 22s - loss: 0.0618 - val_loss: 0.0713
Epoch 10/300
836/836 - 22s - loss: 0.0602 - val_loss: 0.0722
Epoch 11/300
836/836 - 22s - loss: 0.0588 - val_loss: 0.0709
Epoch 12/300
836/836 - 22s - loss: 0.0575 - val_loss: 0.0747
Epoch 13/300
836/836 - 22s - loss: 0.0565 - val_loss: 0.0660
Epoch 14/300
836/836 - 22s - loss: 0.0555 - val_loss: 0.0693
Epoch 15/300
836/836 - 22s - loss: 0.0547 - val_loss: 0.0654
Epoch 16/300
836/836 - 22s - loss: 0.0539 - val_loss: 0.0639
Epoch 17/300
836/836 - 22s - loss: 0.0532 - val_loss: 0.0659
Epoch 18/300
836/836 - 22s - loss: 0.0527 - val_loss: 0.0632
Epoch 19/300
836/836 - 22s - lo

Epoch 23/300
836/836 - 23s - loss: 0.0481 - val_loss: 0.0585
Epoch 24/300
836/836 - 23s - loss: 0.0477 - val_loss: 0.0565
Epoch 25/300
836/836 - 23s - loss: 0.0474 - val_loss: 0.0578
Epoch 26/300
836/836 - 23s - loss: 0.0470 - val_loss: 0.0525
Epoch 27/300
836/836 - 24s - loss: 0.0468 - val_loss: 0.0542
Epoch 28/300
836/836 - 24s - loss: 0.0466 - val_loss: 0.0614
Epoch 29/300
836/836 - 24s - loss: 0.0463 - val_loss: 0.0504
Epoch 30/300
836/836 - 24s - loss: 0.0461 - val_loss: 0.0583
Epoch 31/300
836/836 - 24s - loss: 0.0456 - val_loss: 0.0524
Epoch 32/300
836/836 - 24s - loss: 0.0455 - val_loss: 0.0564
Epoch 33/300
836/836 - 24s - loss: 0.0456 - val_loss: 0.0668
Epoch 34/300
836/836 - 24s - loss: 0.0451 - val_loss: 0.0527
Epoch 35/300
836/836 - 24s - loss: 0.0449 - val_loss: 0.0499
Epoch 36/300
836/836 - 24s - loss: 0.0451 - val_loss: 0.0532
Epoch 37/300
836/836 - 24s - loss: 0.0447 - val_loss: 0.0531
Epoch 38/300
836/836 - 24s - loss: 0.0445 - val_loss: 0.0526
Epoch 39/300
836/836 - 2

Epoch 6/300
836/836 - 23s - loss: 0.0635 - val_loss: 0.0741
Epoch 7/300
836/836 - 23s - loss: 0.0590 - val_loss: 0.0653
Epoch 8/300
836/836 - 23s - loss: 0.0564 - val_loss: 0.0742
Epoch 9/300
836/836 - 23s - loss: 0.0530 - val_loss: 0.0599
Epoch 10/300
836/836 - 23s - loss: 0.0498 - val_loss: 0.0691
Epoch 11/300
836/836 - 23s - loss: 0.0482 - val_loss: 0.0901
Epoch 12/300
836/836 - 23s - loss: 0.0467 - val_loss: 0.0665
Epoch 13/300
836/836 - 23s - loss: 0.0446 - val_loss: 0.0586
Epoch 14/300
836/836 - 23s - loss: 0.0432 - val_loss: 0.0580
Epoch 15/300
836/836 - 23s - loss: 0.0423 - val_loss: 0.0872
Epoch 16/300
836/836 - 23s - loss: 0.0409 - val_loss: 0.0520
Epoch 17/300
836/836 - 23s - loss: 0.0403 - val_loss: 0.0604
Epoch 18/300
836/836 - 23s - loss: 0.0394 - val_loss: 0.0533
Epoch 19/300
836/836 - 23s - loss: 0.0386 - val_loss: 0.0459
Epoch 20/300
836/836 - 24s - loss: 0.0373 - val_loss: 0.0557
Epoch 21/300
836/836 - 24s - loss: 0.0374 - val_loss: 0.0492
Epoch 22/300
836/836 - 24s -

Epoch 53/300
836/836 - 24s - loss: 0.0368 - val_loss: 0.0587
Epoch 54/300
836/836 - 25s - loss: 0.0367 - val_loss: 0.0494
Epoch 55/300
836/836 - 24s - loss: 0.0366 - val_loss: 0.0441
Epoch 56/300
836/836 - 24s - loss: 0.0368 - val_loss: 0.0403
Epoch 57/300
836/836 - 24s - loss: 0.0366 - val_loss: 0.0417
Epoch 58/300
836/836 - 24s - loss: 0.0365 - val_loss: 0.0464
Epoch 59/300
836/836 - 24s - loss: 0.0357 - val_loss: 0.0422
Epoch 60/300
836/836 - 24s - loss: 0.0362 - val_loss: 0.0385
Epoch 61/300
836/836 - 24s - loss: 0.0356 - val_loss: 0.0398
Epoch 62/300
836/836 - 24s - loss: 0.0354 - val_loss: 0.0447
Epoch 63/300
836/836 - 24s - loss: 0.0353 - val_loss: 0.0372
Epoch 64/300
836/836 - 24s - loss: 0.0355 - val_loss: 0.0379
Epoch 65/300
836/836 - 24s - loss: 0.0351 - val_loss: 0.0363
Epoch 66/300
836/836 - 24s - loss: 0.0356 - val_loss: 0.0369
Epoch 67/300
836/836 - 24s - loss: 0.0345 - val_loss: 0.0404
Epoch 68/300
836/836 - 24s - loss: 0.0347 - val_loss: 0.0429
Epoch 69/300
836/836 - 2

## 模型评估

In [86]:
# 评估函数定义
# 异常相关系数
import math
def acc(actual, predicted):
    pred_avg = np.average(predicted)

    act_avg = np.average(actual)
    diff_pred = predicted - pred_avg
    diff_act = actual - act_avg
    numerator = np.mean(np.sum(diff_pred*diff_act, axis=0))
    denominator = math.sqrt(np.mean(np.sum(diff_pred**2, axis=0)) * np.mean(np.sum(diff_act**2, axis=0)))
    ret_val = numerator/denominator
    return (100 * ret_val)


# 查看损失
def look_loss(history):
    plt.figure(figsize=(6.3, 2.5), dpi = 100)
    plt.plot(history.history['loss'], label='training data')
    plt.plot(history.history['val_loss'], label='validation data')
    plt.title('Loss')
    plt.ylabel('MSE Loss')
    plt.xlabel('Number of epochs')
    plt.grid()
    plt.legend(loc="upper right")
    plt.show()


# 模型预测
def all_estimate(best_model,x_test,y_test):
    testPred_1 = best_model.predict(x_test)
    y_test_p = y_test.reshape(-1,1)
    testPred_p = testPred_1.reshape(-1,1)


    rmse = sqrt(mean_squared_error(y_test_p,testPred_p))
    #print('Test RMSE: %.3f' % rmse)
    print('%.3f' % rmse)

   # print('acc:',acc(y_test_p,testPred_p))

    r2 = r2_score(y_test_p,testPred_p)
    #print("R² score: %.4f" % r2)  
    #print("%.4f"% r2)  
    

def respective_estimate(best_model,x_test,y_test):
    re2_list = []
    rmse_list = []
    for i in range(6):
        # 模型预测
        testPred = best_model.predict(x_test[i:i+1])
        a = y_test[i:i+1]
        y_test_p = a.reshape(-1,1)
        testPred_p = testPred.reshape(-1,1)
        r2 = r2_score(y_test_p,testPred_p)
        rmse = sqrt(mean_squared_error(y_test_p,testPred_p))
        re2_list.append(r2)
        rmse_list.append(rmse)    
    return re2_list,rmse_list

In [87]:
#look_loss(history)

### 对指定层进行评估

In [88]:
# print(x_test.shape,y_test.shape)

In [89]:
# best_model = tf.keras.models.load_model('CNN_model_16.h5')
# all_estimate(best_model,x_test,y_test[:,14:15])

### 对所有层进行评估

In [90]:
# 模型预测
for i in range(0,33):
    best_model = tf.keras.models.load_model('D:/codeFile/jupyterDemo/3d_ts/BPNN/model/new_0-2000m_24.6.6/CNN/sali/model4/CNN_sali_'+str(depth[i])+'m.h5',
                                           custom_objects={ 'ChannelAttention_CNN':ChannelAttention_CNN,
                                                           'SpatialAttention_CNN':SpatialAttention_CNN})
    #print(str(depth[i])+'m 测试集：')
    all_estimate(best_model,x_test,y_test[:,i:i+1])
# re2_list,rmse_list = respective_estimate(best_model,x_test,y_test[:,:,:,:,0:5])
# re2_list,rmse_list

0.079
0.086
0.088
0.095
0.120
0.138
0.168
0.186
0.187
0.195
0.199
0.197
0.234
0.242
0.265
0.291
0.281
0.285
0.267
0.275
0.289
0.265
0.261
0.231
0.236
0.270
0.271
0.262
0.276
0.245
0.251
0.239
0.243


# 保存预测结果

In [94]:
# 模型预测
result = [] 
for i in range(0,33):
    best_model = tf.keras.models.load_model('D:/codeFile/jupyterDemo/3d_ts/BPNN/model/new_0-2000m_24.6.6/CNN/sali/model4/CNN_sali_'+str(depth[i])+'m.h5',
                                            custom_objects={ 'ChannelAttention_CNN':ChannelAttention_CNN,
                                                           'SpatialAttention_CNN':SpatialAttention_CNN})
    print('预测第'+str(depth[i])+'m的温度')
    testPred = best_model.predict(x_test)
    result.append(testPred)

预测第5m的温度
预测第10m的温度
预测第15m的温度
预测第20m的温度
预测第30m的温度
预测第40m的温度
预测第50m的温度
预测第60m的温度
预测第70m的温度
预测第80m的温度
预测第90m的温度
预测第100m的温度
预测第125m的温度
预测第150m的温度
预测第175m的温度
预测第200m的温度
预测第225m的温度
预测第250m的温度
预测第275m的温度
预测第300m的温度
预测第350m的温度
预测第400m的温度
预测第500m的温度
预测第600m的温度
预测第700m的温度
预测第800m的温度
预测第900m的温度
预测第1000m的温度
预测第1100m的温度
预测第1300m的温度
预测第1500m的温度
预测第1750m的温度
预测第2000m的温度


In [101]:
# 修改result的形状
result = np.array(result)
print(result.shape)
result = np.transpose(result,(1,2,0))
print(result.shape)
result = result.reshape(-1,33)
print(result.shape)

(33, 1221588, 1)
(1221588, 1, 33)
(1221588, 33)


In [ ]:
#np.save("model/CNN/result/result_CNN_CBAM_30_1000m.npy",result)
# result= np.load("model/CNN/result/result_CNN_CBAM_30_1000m.npy")

## 对预测结果进行反归一化

In [102]:
result.shape,y_test.shape 

((1221588, 33), (1221588, 33))

In [103]:
result_unscaled = scaler_l.inverse_transform(result)
y_test_unscaled = scaler_l.inverse_transform(y_test)

In [104]:
result_unscaled.shape,y_test_unscaled.shape 

((1221588, 33), (1221588, 33))

## 评估反归一化之后的结果

In [ ]:
for i in range(0,33):
    r2 = r2_score(y_test_unscaled[:,i],result_unscaled[:,i])
    print(str(depth[i])+'m R² score:',r2)

In [ ]:
for i in range(0,33):
    mse =  mean_squared_error(y_test_unscaled[:,i],result_unscaled[:,i])
    print(str(depth[i])+'m Test mse: %.3f' % mse)

# 预测结果可视化

## 为归一化之后的预测值和真实值填充nan

In [105]:
len(nan_mask)

161024

In [106]:
### 创建一个形状与nan_masks相同且全是nan的数组
testPred_with_nan = np.full((161024,12*33), np.nan)  
print(testPred_with_nan.shape)

y_test_with_nan = np.full((161024,12*33), np.nan)  
print(y_test_with_nan.shape)

(161024, 396)
(161024, 396)


In [107]:
# 修改result和y_test的形状，用于填充testPred_with_nan和y_test_with_nan
result_unscaled_reshape = result_unscaled.reshape(12,-1,33)
print(result_unscaled_reshape.shape)
result_unscaled_reshape = np.transpose(result_unscaled_reshape,(1,0,2))
print(result_unscaled_reshape.shape)
result_unscaled_reshape = result_unscaled_reshape.reshape(-1,12*33)
print(result_unscaled_reshape.shape)

# ==================================================================== #

y_test_unscaled_reshape = y_test_unscaled.reshape(12,-1,33)
print(y_test_unscaled_reshape.shape)
y_test_unscaled_reshape = np.transpose(y_test_unscaled_reshape,(1,0,2))
print(y_test_unscaled_reshape.shape)
y_test_unscaled_reshape = y_test_unscaled_reshape.reshape(-1,12*33)
print(y_test_unscaled_reshape.shape)

(12, 101799, 33)
(101799, 12, 33)
(101799, 396)
(12, 101799, 33)
(101799, 12, 33)
(101799, 396)


In [ ]:
y_test_unscaled.shape 

In [108]:
num = 0
for i,nanmask in enumerate(nan_mask):   # i从0开始

    if (nanmask ==True): # True 代表没有nan，使用预测值进行填充
        testPred_with_nan[i] = result_unscaled_reshape[num]
        y_test_with_nan[i] = y_test_unscaled_reshape[num]
        num = num + 1
        
print(num)

101799


In [109]:
# 填充好之后将形状修改回来
testPred_with_nan = testPred_with_nan.reshape(272, 592,12,33)
testPred_with_nan = np.transpose(testPred_with_nan,(2,3,0,1))


y_test_with_nan = y_test_with_nan.reshape(272, 592,12,33)
y_test_with_nan = np.transpose(y_test_with_nan,(2,3,0,1))

print(testPred_with_nan.shape,y_test_with_nan.shape)

(12, 33, 272, 592) (12, 33, 272, 592)


# 结果可视化

In [110]:
lat = []
lon = []
time = []

In [111]:
lon.append(data_ssh['lon'][4:-4].data)
lat.append(data_ssh['lat'][4:-4].data)

In [112]:
len(lat),len(lat[0]),len(lon),len(lon[0])

(1, 272, 1, 592)

In [113]:
start_year = 2015
end_year =2015
time = []
date_range = pd.date_range(datetime(start_year,1,1),datetime(end_year+1,1,1),freq='1M')
len(date_range)
for i in range(len(date_range)):
    time.append(i)

In [ ]:
# new_NC.close()

In [116]:
new_NC = nc.Dataset("D:/codeFile/jupyterDemo/3d_ts/BPNN/model/new_0-2000m_24.6.6/CNN/sali/result4/result5_2000m_CNN_CBAM.nc",
                    'w', format='NETCDF4')

'''
定义维度，后一个参数表示维度的长度，因为是合并的同一个产品的数据，所以是统一
的,注意维度的长度一定要和读入的数据匹配
'''

new_NC.createDimension('lat', len(lat[0]))
new_NC.createDimension('lon', len(lon[0]))

new_NC.createDimension('depth', 33)
new_NC.createDimension('time', len(time))

new_NC.createVariable('lat', 'f', ("lat"))
new_NC.createVariable('lon', 'f', ("lon"))
new_NC.createVariable('depth', 'f', ("depth"))
new_NC.createVariable('Observed_SS', 'f',("time","depth","lat","lon"))
new_NC.createVariable('Estimated_SS', 'f',("time","depth","lat","lon"))

time_var = new_NC.createVariable('time', 'f4',("time"))
time_var.units = 'months since 2015-01-15'
time_var.long_name = 'Months in Monthly Means'
time_var.axis = 'T'

#向变量中填充数据
new_NC.variables['lat'][:] = lat[0]
new_NC.variables['lon'][:] = lon[0]
new_NC.variables['time'][:] = np.array(time)
new_NC.variables['depth'][:] = depth

new_NC.variables['Observed_SS'][:]=np.array(y_test_with_nan)
new_NC.variables['Estimated_SS'][:]=np.array(testPred_with_nan)


#最后记得关闭文件
new_NC.close()
